In [6]:
import web3, json, requests, time, datetime, math, os
from web3 import Web3
from pprint import pprint
import numpy as np
import pandas as pd
from scipy.stats import norm
from pandas_datareader import data as pdr
from functools import reduce

# Load dotenv and set INFURA_API_KEY
%reload_ext dotenv
%dotenv
INFURA_API_KEY = os.getenv("INFURA_API_KEY")


# Setup web3, addresses and ABIs
w3 = Web3(Web3.HTTPProvider(f'https://mainnet.infura.io/v3/{INFURA_API_KEY}'))
compound_ceth_abi_string = "[{\"constant\":true,\"inputs\":[],\"name\":\"name\",\"outputs\":[{\"name\":\"\",\"type\":\"string\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"spender\",\"type\":\"address\"},{\"name\":\"amount\",\"type\":\"uint256\"}],\"name\":\"approve\",\"outputs\":[{\"name\":\"\",\"type\":\"bool\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[],\"name\":\"mint\",\"outputs\":[],\"payable\":true,\"stateMutability\":\"payable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"reserveFactorMantissa\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"account\",\"type\":\"address\"}],\"name\":\"borrowBalanceCurrent\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"totalSupply\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"exchangeRateStored\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"src\",\"type\":\"address\"},{\"name\":\"dst\",\"type\":\"address\"},{\"name\":\"amount\",\"type\":\"uint256\"}],\"name\":\"transferFrom\",\"outputs\":[{\"name\":\"\",\"type\":\"bool\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"pendingAdmin\",\"outputs\":[{\"name\":\"\",\"type\":\"address\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"decimals\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"owner\",\"type\":\"address\"}],\"name\":\"balanceOfUnderlying\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"getCash\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"newComptroller\",\"type\":\"address\"}],\"name\":\"_setComptroller\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"totalBorrows\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[],\"name\":\"repayBorrow\",\"outputs\":[],\"payable\":true,\"stateMutability\":\"payable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"comptroller\",\"outputs\":[{\"name\":\"\",\"type\":\"address\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"reduceAmount\",\"type\":\"uint256\"}],\"name\":\"_reduceReserves\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"initialExchangeRateMantissa\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"accrualBlockNumber\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"name\":\"owner\",\"type\":\"address\"}],\"name\":\"balanceOf\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[],\"name\":\"totalBorrowsCurrent\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"redeemAmount\",\"type\":\"uint256\"}],\"name\":\"redeemUnderlying\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"totalReserves\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"symbol\",\"outputs\":[{\"name\":\"\",\"type\":\"string\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"name\":\"account\",\"type\":\"address\"}],\"name\":\"borrowBalanceStored\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[],\"name\":\"accrueInterest\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"dst\",\"type\":\"address\"},{\"name\":\"amount\",\"type\":\"uint256\"}],\"name\":\"transfer\",\"outputs\":[{\"name\":\"\",\"type\":\"bool\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"borrowIndex\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"borrower\",\"type\":\"address\"},{\"name\":\"cTokenCollateral\",\"type\":\"address\"}],\"name\":\"liquidateBorrow\",\"outputs\":[],\"payable\":true,\"stateMutability\":\"payable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"supplyRatePerBlock\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"liquidator\",\"type\":\"address\"},{\"name\":\"borrower\",\"type\":\"address\"},{\"name\":\"seizeTokens\",\"type\":\"uint256\"}],\"name\":\"seize\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"newPendingAdmin\",\"type\":\"address\"}],\"name\":\"_setPendingAdmin\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[],\"name\":\"exchangeRateCurrent\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"name\":\"account\",\"type\":\"address\"}],\"name\":\"getAccountSnapshot\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"},{\"name\":\"\",\"type\":\"uint256\"},{\"name\":\"\",\"type\":\"uint256\"},{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"borrowAmount\",\"type\":\"uint256\"}],\"name\":\"borrow\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"redeemTokens\",\"type\":\"uint256\"}],\"name\":\"redeem\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"name\":\"owner\",\"type\":\"address\"},{\"name\":\"spender\",\"type\":\"address\"}],\"name\":\"allowance\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"borrower\",\"type\":\"address\"}],\"name\":\"repayBorrowBehalf\",\"outputs\":[],\"payable\":true,\"stateMutability\":\"payable\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[],\"name\":\"_acceptAdmin\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"newInterestRateModel\",\"type\":\"address\"}],\"name\":\"_setInterestRateModel\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"interestRateModel\",\"outputs\":[{\"name\":\"\",\"type\":\"address\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"admin\",\"outputs\":[{\"name\":\"\",\"type\":\"address\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"borrowRatePerBlock\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"newReserveFactorMantissa\",\"type\":\"uint256\"}],\"name\":\"_setReserveFactor\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"isCToken\",\"outputs\":[{\"name\":\"\",\"type\":\"bool\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[{\"name\":\"comptroller_\",\"type\":\"address\"},{\"name\":\"interestRateModel_\",\"type\":\"address\"},{\"name\":\"initialExchangeRateMantissa_\",\"type\":\"uint256\"},{\"name\":\"name_\",\"type\":\"string\"},{\"name\":\"symbol_\",\"type\":\"string\"},{\"name\":\"decimals_\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"constructor\"},{\"payable\":true,\"stateMutability\":\"payable\",\"type\":\"fallback\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"name\":\"interestAccumulated\",\"type\":\"uint256\"},{\"indexed\":false,\"name\":\"borrowIndex\",\"type\":\"uint256\"},{\"indexed\":false,\"name\":\"totalBorrows\",\"type\":\"uint256\"}],\"name\":\"AccrueInterest\",\"type\":\"event\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"name\":\"minter\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"mintAmount\",\"type\":\"uint256\"},{\"indexed\":false,\"name\":\"mintTokens\",\"type\":\"uint256\"}],\"name\":\"Mint\",\"type\":\"event\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"name\":\"redeemer\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"redeemAmount\",\"type\":\"uint256\"},{\"indexed\":false,\"name\":\"redeemTokens\",\"type\":\"uint256\"}],\"name\":\"Redeem\",\"type\":\"event\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"name\":\"borrower\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"borrowAmount\",\"type\":\"uint256\"},{\"indexed\":false,\"name\":\"accountBorrows\",\"type\":\"uint256\"},{\"indexed\":false,\"name\":\"totalBorrows\",\"type\":\"uint256\"}],\"name\":\"Borrow\",\"type\":\"event\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"name\":\"payer\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"borrower\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"repayAmount\",\"type\":\"uint256\"},{\"indexed\":false,\"name\":\"accountBorrows\",\"type\":\"uint256\"},{\"indexed\":false,\"name\":\"totalBorrows\",\"type\":\"uint256\"}],\"name\":\"RepayBorrow\",\"type\":\"event\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"name\":\"liquidator\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"borrower\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"repayAmount\",\"type\":\"uint256\"},{\"indexed\":false,\"name\":\"cTokenCollateral\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"seizeTokens\",\"type\":\"uint256\"}],\"name\":\"LiquidateBorrow\",\"type\":\"event\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"name\":\"oldPendingAdmin\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"newPendingAdmin\",\"type\":\"address\"}],\"name\":\"NewPendingAdmin\",\"type\":\"event\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"name\":\"oldAdmin\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"newAdmin\",\"type\":\"address\"}],\"name\":\"NewAdmin\",\"type\":\"event\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"name\":\"oldComptroller\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"newComptroller\",\"type\":\"address\"}],\"name\":\"NewComptroller\",\"type\":\"event\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"name\":\"oldInterestRateModel\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"newInterestRateModel\",\"type\":\"address\"}],\"name\":\"NewMarketInterestRateModel\",\"type\":\"event\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"name\":\"oldReserveFactorMantissa\",\"type\":\"uint256\"},{\"indexed\":false,\"name\":\"newReserveFactorMantissa\",\"type\":\"uint256\"}],\"name\":\"NewReserveFactor\",\"type\":\"event\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"name\":\"admin\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"reduceAmount\",\"type\":\"uint256\"},{\"indexed\":false,\"name\":\"newTotalReserves\",\"type\":\"uint256\"}],\"name\":\"ReservesReduced\",\"type\":\"event\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"name\":\"error\",\"type\":\"uint256\"},{\"indexed\":false,\"name\":\"info\",\"type\":\"uint256\"},{\"indexed\":false,\"name\":\"detail\",\"type\":\"uint256\"}],\"name\":\"Failure\",\"type\":\"event\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":true,\"name\":\"from\",\"type\":\"address\"},{\"indexed\":true,\"name\":\"to\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"amount\",\"type\":\"uint256\"}],\"name\":\"Transfer\",\"type\":\"event\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":true,\"name\":\"owner\",\"type\":\"address\"},{\"indexed\":true,\"name\":\"spender\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"amount\",\"type\":\"uint256\"}],\"name\":\"Approval\",\"type\":\"event\"}]"
erc20_abi_string = "[{\"constant\":true,\"inputs\":[],\"name\":\"name\",\"outputs\":[{\"name\":\"\",\"type\":\"string\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"spender\",\"type\":\"address\"},{\"name\":\"amount\",\"type\":\"uint256\"}],\"name\":\"approve\",\"outputs\":[{\"name\":\"\",\"type\":\"bool\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"repayAmount\",\"type\":\"uint256\"}],\"name\":\"repayBorrow\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"reserveFactorMantissa\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"account\",\"type\":\"address\"}],\"name\":\"borrowBalanceCurrent\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"totalSupply\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"exchangeRateStored\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"src\",\"type\":\"address\"},{\"name\":\"dst\",\"type\":\"address\"},{\"name\":\"amount\",\"type\":\"uint256\"}],\"name\":\"transferFrom\",\"outputs\":[{\"name\":\"\",\"type\":\"bool\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"borrower\",\"type\":\"address\"},{\"name\":\"repayAmount\",\"type\":\"uint256\"}],\"name\":\"repayBorrowBehalf\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"pendingAdmin\",\"outputs\":[{\"name\":\"\",\"type\":\"address\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"decimals\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"owner\",\"type\":\"address\"}],\"name\":\"balanceOfUnderlying\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"getCash\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"newComptroller\",\"type\":\"address\"}],\"name\":\"_setComptroller\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"totalBorrows\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"comptroller\",\"outputs\":[{\"name\":\"\",\"type\":\"address\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"reduceAmount\",\"type\":\"uint256\"}],\"name\":\"_reduceReserves\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"initialExchangeRateMantissa\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"accrualBlockNumber\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"underlying\",\"outputs\":[{\"name\":\"\",\"type\":\"address\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"name\":\"owner\",\"type\":\"address\"}],\"name\":\"balanceOf\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[],\"name\":\"totalBorrowsCurrent\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"redeemAmount\",\"type\":\"uint256\"}],\"name\":\"redeemUnderlying\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"totalReserves\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"symbol\",\"outputs\":[{\"name\":\"\",\"type\":\"string\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"name\":\"account\",\"type\":\"address\"}],\"name\":\"borrowBalanceStored\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"mintAmount\",\"type\":\"uint256\"}],\"name\":\"mint\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[],\"name\":\"accrueInterest\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"dst\",\"type\":\"address\"},{\"name\":\"amount\",\"type\":\"uint256\"}],\"name\":\"transfer\",\"outputs\":[{\"name\":\"\",\"type\":\"bool\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"borrowIndex\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"supplyRatePerBlock\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"liquidator\",\"type\":\"address\"},{\"name\":\"borrower\",\"type\":\"address\"},{\"name\":\"seizeTokens\",\"type\":\"uint256\"}],\"name\":\"seize\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"newPendingAdmin\",\"type\":\"address\"}],\"name\":\"_setPendingAdmin\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[],\"name\":\"exchangeRateCurrent\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"name\":\"account\",\"type\":\"address\"}],\"name\":\"getAccountSnapshot\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"},{\"name\":\"\",\"type\":\"uint256\"},{\"name\":\"\",\"type\":\"uint256\"},{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"borrowAmount\",\"type\":\"uint256\"}],\"name\":\"borrow\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"redeemTokens\",\"type\":\"uint256\"}],\"name\":\"redeem\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"name\":\"owner\",\"type\":\"address\"},{\"name\":\"spender\",\"type\":\"address\"}],\"name\":\"allowance\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[],\"name\":\"_acceptAdmin\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"newInterestRateModel\",\"type\":\"address\"}],\"name\":\"_setInterestRateModel\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"interestRateModel\",\"outputs\":[{\"name\":\"\",\"type\":\"address\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"borrower\",\"type\":\"address\"},{\"name\":\"repayAmount\",\"type\":\"uint256\"},{\"name\":\"cTokenCollateral\",\"type\":\"address\"}],\"name\":\"liquidateBorrow\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"admin\",\"outputs\":[{\"name\":\"\",\"type\":\"address\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"borrowRatePerBlock\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"newReserveFactorMantissa\",\"type\":\"uint256\"}],\"name\":\"_setReserveFactor\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"isCToken\",\"outputs\":[{\"name\":\"\",\"type\":\"bool\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[{\"name\":\"underlying_\",\"type\":\"address\"},{\"name\":\"comptroller_\",\"type\":\"address\"},{\"name\":\"interestRateModel_\",\"type\":\"address\"},{\"name\":\"initialExchangeRateMantissa_\",\"type\":\"uint256\"},{\"name\":\"name_\",\"type\":\"string\"},{\"name\":\"symbol_\",\"type\":\"string\"},{\"name\":\"decimals_\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"constructor\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"name\":\"interestAccumulated\",\"type\":\"uint256\"},{\"indexed\":false,\"name\":\"borrowIndex\",\"type\":\"uint256\"},{\"indexed\":false,\"name\":\"totalBorrows\",\"type\":\"uint256\"}],\"name\":\"AccrueInterest\",\"type\":\"event\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"name\":\"minter\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"mintAmount\",\"type\":\"uint256\"},{\"indexed\":false,\"name\":\"mintTokens\",\"type\":\"uint256\"}],\"name\":\"Mint\",\"type\":\"event\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"name\":\"redeemer\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"redeemAmount\",\"type\":\"uint256\"},{\"indexed\":false,\"name\":\"redeemTokens\",\"type\":\"uint256\"}],\"name\":\"Redeem\",\"type\":\"event\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"name\":\"borrower\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"borrowAmount\",\"type\":\"uint256\"},{\"indexed\":false,\"name\":\"accountBorrows\",\"type\":\"uint256\"},{\"indexed\":false,\"name\":\"totalBorrows\",\"type\":\"uint256\"}],\"name\":\"Borrow\",\"type\":\"event\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"name\":\"payer\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"borrower\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"repayAmount\",\"type\":\"uint256\"},{\"indexed\":false,\"name\":\"accountBorrows\",\"type\":\"uint256\"},{\"indexed\":false,\"name\":\"totalBorrows\",\"type\":\"uint256\"}],\"name\":\"RepayBorrow\",\"type\":\"event\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"name\":\"liquidator\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"borrower\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"repayAmount\",\"type\":\"uint256\"},{\"indexed\":false,\"name\":\"cTokenCollateral\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"seizeTokens\",\"type\":\"uint256\"}],\"name\":\"LiquidateBorrow\",\"type\":\"event\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"name\":\"oldPendingAdmin\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"newPendingAdmin\",\"type\":\"address\"}],\"name\":\"NewPendingAdmin\",\"type\":\"event\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"name\":\"oldAdmin\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"newAdmin\",\"type\":\"address\"}],\"name\":\"NewAdmin\",\"type\":\"event\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"name\":\"oldComptroller\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"newComptroller\",\"type\":\"address\"}],\"name\":\"NewComptroller\",\"type\":\"event\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"name\":\"oldInterestRateModel\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"newInterestRateModel\",\"type\":\"address\"}],\"name\":\"NewMarketInterestRateModel\",\"type\":\"event\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"name\":\"oldReserveFactorMantissa\",\"type\":\"uint256\"},{\"indexed\":false,\"name\":\"newReserveFactorMantissa\",\"type\":\"uint256\"}],\"name\":\"NewReserveFactor\",\"type\":\"event\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"name\":\"admin\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"reduceAmount\",\"type\":\"uint256\"},{\"indexed\":false,\"name\":\"newTotalReserves\",\"type\":\"uint256\"}],\"name\":\"ReservesReduced\",\"type\":\"event\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"name\":\"error\",\"type\":\"uint256\"},{\"indexed\":false,\"name\":\"info\",\"type\":\"uint256\"},{\"indexed\":false,\"name\":\"detail\",\"type\":\"uint256\"}],\"name\":\"Failure\",\"type\":\"event\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":true,\"name\":\"from\",\"type\":\"address\"},{\"indexed\":true,\"name\":\"to\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"amount\",\"type\":\"uint256\"}],\"name\":\"Transfer\",\"type\":\"event\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":true,\"name\":\"owner\",\"type\":\"address\"},{\"indexed\":true,\"name\":\"spender\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"amount\",\"type\":\"uint256\"}],\"name\":\"Approval\",\"type\":\"event\"}]"
dydx_abi_string = "[{\"constant\":false,\"inputs\":[{\"name\":\"marketId\",\"type\":\"uint256\"},{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"spreadPremium\",\"type\":\"tuple\"}],\"name\":\"ownerSetSpreadPremium\",\"outputs\":[],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"name\":\"operator\",\"type\":\"address\"}],\"name\":\"getIsGlobalOperator\",\"outputs\":[{\"name\":\"\",\"type\":\"bool\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"name\":\"marketId\",\"type\":\"uint256\"}],\"name\":\"getMarketTokenAddress\",\"outputs\":[{\"name\":\"\",\"type\":\"address\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"marketId\",\"type\":\"uint256\"},{\"name\":\"interestSetter\",\"type\":\"address\"}],\"name\":\"ownerSetInterestSetter\",\"outputs\":[],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"components\":[{\"name\":\"owner\",\"type\":\"address\"},{\"name\":\"number\",\"type\":\"uint256\"}],\"name\":\"account\",\"type\":\"tuple\"}],\"name\":\"getAccountValues\",\"outputs\":[{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"\",\"type\":\"tuple\"},{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"\",\"type\":\"tuple\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"name\":\"marketId\",\"type\":\"uint256\"}],\"name\":\"getMarketPriceOracle\",\"outputs\":[{\"name\":\"\",\"type\":\"address\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"name\":\"marketId\",\"type\":\"uint256\"}],\"name\":\"getMarketInterestSetter\",\"outputs\":[{\"name\":\"\",\"type\":\"address\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"name\":\"marketId\",\"type\":\"uint256\"}],\"name\":\"getMarketSpreadPremium\",\"outputs\":[{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"\",\"type\":\"tuple\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"getNumMarkets\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"token\",\"type\":\"address\"},{\"name\":\"recipient\",\"type\":\"address\"}],\"name\":\"ownerWithdrawUnsupportedTokens\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"minBorrowedValue\",\"type\":\"tuple\"}],\"name\":\"ownerSetMinBorrowedValue\",\"outputs\":[],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"spread\",\"type\":\"tuple\"}],\"name\":\"ownerSetLiquidationSpread\",\"outputs\":[],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"earningsRate\",\"type\":\"tuple\"}],\"name\":\"ownerSetEarningsRate\",\"outputs\":[],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"name\":\"owner\",\"type\":\"address\"},{\"name\":\"operator\",\"type\":\"address\"}],\"name\":\"getIsLocalOperator\",\"outputs\":[{\"name\":\"\",\"type\":\"bool\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"components\":[{\"name\":\"owner\",\"type\":\"address\"},{\"name\":\"number\",\"type\":\"uint256\"}],\"name\":\"account\",\"type\":\"tuple\"},{\"name\":\"marketId\",\"type\":\"uint256\"}],\"name\":\"getAccountPar\",\"outputs\":[{\"components\":[{\"name\":\"sign\",\"type\":\"bool\"},{\"name\":\"value\",\"type\":\"uint128\"}],\"name\":\"\",\"type\":\"tuple\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"marketId\",\"type\":\"uint256\"},{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"marginPremium\",\"type\":\"tuple\"}],\"name\":\"ownerSetMarginPremium\",\"outputs\":[],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"getMarginRatio\",\"outputs\":[{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"\",\"type\":\"tuple\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"name\":\"marketId\",\"type\":\"uint256\"}],\"name\":\"getMarketCurrentIndex\",\"outputs\":[{\"components\":[{\"name\":\"borrow\",\"type\":\"uint96\"},{\"name\":\"supply\",\"type\":\"uint96\"},{\"name\":\"lastUpdate\",\"type\":\"uint32\"}],\"name\":\"\",\"type\":\"tuple\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"name\":\"marketId\",\"type\":\"uint256\"}],\"name\":\"getMarketIsClosing\",\"outputs\":[{\"name\":\"\",\"type\":\"bool\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"getRiskParams\",\"outputs\":[{\"components\":[{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"marginRatio\",\"type\":\"tuple\"},{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"liquidationSpread\",\"type\":\"tuple\"},{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"earningsRate\",\"type\":\"tuple\"},{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"minBorrowedValue\",\"type\":\"tuple\"}],\"name\":\"\",\"type\":\"tuple\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"components\":[{\"name\":\"owner\",\"type\":\"address\"},{\"name\":\"number\",\"type\":\"uint256\"}],\"name\":\"account\",\"type\":\"tuple\"}],\"name\":\"getAccountBalances\",\"outputs\":[{\"name\":\"\",\"type\":\"address[]\"},{\"components\":[{\"name\":\"sign\",\"type\":\"bool\"},{\"name\":\"value\",\"type\":\"uint128\"}],\"name\":\"\",\"type\":\"tuple[]\"},{\"components\":[{\"name\":\"sign\",\"type\":\"bool\"},{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"\",\"type\":\"tuple[]\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[],\"name\":\"renounceOwnership\",\"outputs\":[],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"getMinBorrowedValue\",\"outputs\":[{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"\",\"type\":\"tuple\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"components\":[{\"name\":\"operator\",\"type\":\"address\"},{\"name\":\"trusted\",\"type\":\"bool\"}],\"name\":\"args\",\"type\":\"tuple[]\"}],\"name\":\"setOperators\",\"outputs\":[],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"name\":\"marketId\",\"type\":\"uint256\"}],\"name\":\"getMarketPrice\",\"outputs\":[{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"\",\"type\":\"tuple\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"owner\",\"outputs\":[{\"name\":\"\",\"type\":\"address\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"isOwner\",\"outputs\":[{\"name\":\"\",\"type\":\"bool\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"marketId\",\"type\":\"uint256\"},{\"name\":\"recipient\",\"type\":\"address\"}],\"name\":\"ownerWithdrawExcessTokens\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"token\",\"type\":\"address\"},{\"name\":\"priceOracle\",\"type\":\"address\"},{\"name\":\"interestSetter\",\"type\":\"address\"},{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"marginPremium\",\"type\":\"tuple\"},{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"spreadPremium\",\"type\":\"tuple\"}],\"name\":\"ownerAddMarket\",\"outputs\":[],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"components\":[{\"name\":\"owner\",\"type\":\"address\"},{\"name\":\"number\",\"type\":\"uint256\"}],\"name\":\"accounts\",\"type\":\"tuple[]\"},{\"components\":[{\"name\":\"actionType\",\"type\":\"uint8\"},{\"name\":\"accountId\",\"type\":\"uint256\"},{\"components\":[{\"name\":\"sign\",\"type\":\"bool\"},{\"name\":\"denomination\",\"type\":\"uint8\"},{\"name\":\"ref\",\"type\":\"uint8\"},{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"amount\",\"type\":\"tuple\"},{\"name\":\"primaryMarketId\",\"type\":\"uint256\"},{\"name\":\"secondaryMarketId\",\"type\":\"uint256\"},{\"name\":\"otherAddress\",\"type\":\"address\"},{\"name\":\"otherAccountId\",\"type\":\"uint256\"},{\"name\":\"data\",\"type\":\"bytes\"}],\"name\":\"actions\",\"type\":\"tuple[]\"}],\"name\":\"operate\",\"outputs\":[],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"name\":\"marketId\",\"type\":\"uint256\"}],\"name\":\"getMarketWithInfo\",\"outputs\":[{\"components\":[{\"name\":\"token\",\"type\":\"address\"},{\"components\":[{\"name\":\"borrow\",\"type\":\"uint128\"},{\"name\":\"supply\",\"type\":\"uint128\"}],\"name\":\"totalPar\",\"type\":\"tuple\"},{\"components\":[{\"name\":\"borrow\",\"type\":\"uint96\"},{\"name\":\"supply\",\"type\":\"uint96\"},{\"name\":\"lastUpdate\",\"type\":\"uint32\"}],\"name\":\"index\",\"type\":\"tuple\"},{\"name\":\"priceOracle\",\"type\":\"address\"},{\"name\":\"interestSetter\",\"type\":\"address\"},{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"marginPremium\",\"type\":\"tuple\"},{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"spreadPremium\",\"type\":\"tuple\"},{\"name\":\"isClosing\",\"type\":\"bool\"}],\"name\":\"\",\"type\":\"tuple\"},{\"components\":[{\"name\":\"borrow\",\"type\":\"uint96\"},{\"name\":\"supply\",\"type\":\"uint96\"},{\"name\":\"lastUpdate\",\"type\":\"uint32\"}],\"name\":\"\",\"type\":\"tuple\"},{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"\",\"type\":\"tuple\"},{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"\",\"type\":\"tuple\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"ratio\",\"type\":\"tuple\"}],\"name\":\"ownerSetMarginRatio\",\"outputs\":[],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"getLiquidationSpread\",\"outputs\":[{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"\",\"type\":\"tuple\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"components\":[{\"name\":\"owner\",\"type\":\"address\"},{\"name\":\"number\",\"type\":\"uint256\"}],\"name\":\"account\",\"type\":\"tuple\"},{\"name\":\"marketId\",\"type\":\"uint256\"}],\"name\":\"getAccountWei\",\"outputs\":[{\"components\":[{\"name\":\"sign\",\"type\":\"bool\"},{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"\",\"type\":\"tuple\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"name\":\"marketId\",\"type\":\"uint256\"}],\"name\":\"getMarketTotalPar\",\"outputs\":[{\"components\":[{\"name\":\"borrow\",\"type\":\"uint128\"},{\"name\":\"supply\",\"type\":\"uint128\"}],\"name\":\"\",\"type\":\"tuple\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"name\":\"heldMarketId\",\"type\":\"uint256\"},{\"name\":\"owedMarketId\",\"type\":\"uint256\"}],\"name\":\"getLiquidationSpreadForPair\",\"outputs\":[{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"\",\"type\":\"tuple\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"name\":\"marketId\",\"type\":\"uint256\"}],\"name\":\"getNumExcessTokens\",\"outputs\":[{\"components\":[{\"name\":\"sign\",\"type\":\"bool\"},{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"\",\"type\":\"tuple\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"name\":\"marketId\",\"type\":\"uint256\"}],\"name\":\"getMarketCachedIndex\",\"outputs\":[{\"components\":[{\"name\":\"borrow\",\"type\":\"uint96\"},{\"name\":\"supply\",\"type\":\"uint96\"},{\"name\":\"lastUpdate\",\"type\":\"uint32\"}],\"name\":\"\",\"type\":\"tuple\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"components\":[{\"name\":\"owner\",\"type\":\"address\"},{\"name\":\"number\",\"type\":\"uint256\"}],\"name\":\"account\",\"type\":\"tuple\"}],\"name\":\"getAccountStatus\",\"outputs\":[{\"name\":\"\",\"type\":\"uint8\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"getEarningsRate\",\"outputs\":[{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"\",\"type\":\"tuple\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"marketId\",\"type\":\"uint256\"},{\"name\":\"priceOracle\",\"type\":\"address\"}],\"name\":\"ownerSetPriceOracle\",\"outputs\":[],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"getRiskLimits\",\"outputs\":[{\"components\":[{\"name\":\"marginRatioMax\",\"type\":\"uint64\"},{\"name\":\"liquidationSpreadMax\",\"type\":\"uint64\"},{\"name\":\"earningsRateMax\",\"type\":\"uint64\"},{\"name\":\"marginPremiumMax\",\"type\":\"uint64\"},{\"name\":\"spreadPremiumMax\",\"type\":\"uint64\"},{\"name\":\"minBorrowedValueMax\",\"type\":\"uint128\"}],\"name\":\"\",\"type\":\"tuple\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"name\":\"marketId\",\"type\":\"uint256\"}],\"name\":\"getMarket\",\"outputs\":[{\"components\":[{\"name\":\"token\",\"type\":\"address\"},{\"components\":[{\"name\":\"borrow\",\"type\":\"uint128\"},{\"name\":\"supply\",\"type\":\"uint128\"}],\"name\":\"totalPar\",\"type\":\"tuple\"},{\"components\":[{\"name\":\"borrow\",\"type\":\"uint96\"},{\"name\":\"supply\",\"type\":\"uint96\"},{\"name\":\"lastUpdate\",\"type\":\"uint32\"}],\"name\":\"index\",\"type\":\"tuple\"},{\"name\":\"priceOracle\",\"type\":\"address\"},{\"name\":\"interestSetter\",\"type\":\"address\"},{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"marginPremium\",\"type\":\"tuple\"},{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"spreadPremium\",\"type\":\"tuple\"},{\"name\":\"isClosing\",\"type\":\"bool\"}],\"name\":\"\",\"type\":\"tuple\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"marketId\",\"type\":\"uint256\"},{\"name\":\"isClosing\",\"type\":\"bool\"}],\"name\":\"ownerSetIsClosing\",\"outputs\":[],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"operator\",\"type\":\"address\"},{\"name\":\"approved\",\"type\":\"bool\"}],\"name\":\"ownerSetGlobalOperator\",\"outputs\":[],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[{\"name\":\"newOwner\",\"type\":\"address\"}],\"name\":\"transferOwnership\",\"outputs\":[],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"components\":[{\"name\":\"owner\",\"type\":\"address\"},{\"name\":\"number\",\"type\":\"uint256\"}],\"name\":\"account\",\"type\":\"tuple\"}],\"name\":\"getAdjustedAccountValues\",\"outputs\":[{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"\",\"type\":\"tuple\"},{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"\",\"type\":\"tuple\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"name\":\"marketId\",\"type\":\"uint256\"}],\"name\":\"getMarketMarginPremium\",\"outputs\":[{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"\",\"type\":\"tuple\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"name\":\"marketId\",\"type\":\"uint256\"}],\"name\":\"getMarketInterestRate\",\"outputs\":[{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"\",\"type\":\"tuple\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[{\"components\":[{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"marginRatio\",\"type\":\"tuple\"},{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"liquidationSpread\",\"type\":\"tuple\"},{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"earningsRate\",\"type\":\"tuple\"},{\"components\":[{\"name\":\"value\",\"type\":\"uint256\"}],\"name\":\"minBorrowedValue\",\"type\":\"tuple\"}],\"name\":\"riskParams\",\"type\":\"tuple\"},{\"components\":[{\"name\":\"marginRatioMax\",\"type\":\"uint64\"},{\"name\":\"liquidationSpreadMax\",\"type\":\"uint64\"},{\"name\":\"earningsRateMax\",\"type\":\"uint64\"},{\"name\":\"marginPremiumMax\",\"type\":\"uint64\"},{\"name\":\"spreadPremiumMax\",\"type\":\"uint64\"},{\"name\":\"minBorrowedValueMax\",\"type\":\"uint128\"}],\"name\":\"riskLimits\",\"type\":\"tuple\"}],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"constructor\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":true,\"name\":\"owner\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"operator\",\"type\":\"address\"},{\"indexed\":false,\"name\":\"trusted\",\"type\":\"bool\"}],\"name\":\"LogOperatorSet\",\"type\":\"event\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":true,\"name\":\"previousOwner\",\"type\":\"address\"},{\"indexed\":true,\"name\":\"newOwner\",\"type\":\"address\"}],\"name\":\"OwnershipTransferred\",\"type\":\"event\"}]"
fulcrum_itoken_abi = '[{"constant":true,"inputs":[],"name":"name","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x06fdde03"},{"constant":false,"inputs":[{"name":"_spender","type":"address"},{"name":"_value","type":"uint256"}],"name":"approve","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function","signature":"0x095ea7b3"},{"constant":true,"inputs":[],"name":"burntTokenReserved","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x0c4925fd"},{"constant":true,"inputs":[],"name":"totalSupply","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x18160ddd"},{"constant":true,"inputs":[],"name":"initialPrice","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x1d0806ae"},{"constant":true,"inputs":[],"name":"baseRate","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x1f68f20a"},{"constant":true,"inputs":[],"name":"totalAssetBorrow","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x20f6d07c"},{"constant":true,"inputs":[{"name":"","type":"bytes32"}],"name":"loanOrderData","outputs":[{"name":"loanOrderHash","type":"bytes32"},{"name":"leverageAmount","type":"uint256"},{"name":"initialMarginAmount","type":"uint256"},{"name":"maintenanceMarginAmount","type":"uint256"},{"name":"maxDurationUnixTimestampSec","type":"uint256"},{"name":"index","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x2515aacd"},{"constant":true,"inputs":[],"name":"decimals","outputs":[{"name":"","type":"uint8"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x313ce567"},{"constant":true,"inputs":[],"name":"rateMultiplier","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x330691ac"},{"constant":true,"inputs":[],"name":"wethContract","outputs":[{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x4780eac1"},{"constant":false,"inputs":[{"name":"_spender","type":"address"},{"name":"_subtractedValue","type":"uint256"}],"name":"decreaseApproval","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function","signature":"0x66188463"},{"constant":true,"inputs":[{"name":"_owner","type":"address"}],"name":"balanceOf","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x70a08231"},{"constant":true,"inputs":[],"name":"tokenizedRegistry","outputs":[{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x736ee3d3"},{"constant":true,"inputs":[{"name":"","type":"uint256"}],"name":"burntTokenReserveList","outputs":[{"name":"lender","type":"address"},{"name":"amount","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x7866c6c1"},{"constant":true,"inputs":[],"name":"loanTokenAddress","outputs":[{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x797bf385"},{"constant":true,"inputs":[],"name":"checkpointSupply","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x7b7933b4"},{"constant":true,"inputs":[],"name":"bZxVault","outputs":[{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x894ca308"},{"constant":true,"inputs":[],"name":"owner","outputs":[{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x8da5cb5b"},{"constant":true,"inputs":[],"name":"symbol","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x95d89b41"},{"constant":true,"inputs":[],"name":"bZxOracle","outputs":[{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x96c7871b"},{"constant":true,"inputs":[],"name":"bZxContract","outputs":[{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x995363d3"},{"constant":true,"inputs":[{"name":"","type":"uint256"}],"name":"leverageList","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x9b3a54d1"},{"constant":false,"inputs":[{"name":"_spender","type":"address"},{"name":"_addedValue","type":"uint256"}],"name":"increaseApproval","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function","signature":"0xd73dd623"},{"constant":true,"inputs":[],"name":"spreadMultiplier","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function","signature":"0xd84d2a47"},{"constant":true,"inputs":[{"name":"_owner","type":"address"},{"name":"_spender","type":"address"}],"name":"allowance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function","signature":"0xdd62ed3e"},{"constant":false,"inputs":[{"name":"_newOwner","type":"address"}],"name":"transferOwnership","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function","signature":"0xf2fde38b"},{"constant":true,"inputs":[{"name":"","type":"address"}],"name":"burntTokenReserveListIndex","outputs":[{"name":"index","type":"uint256"},{"name":"isSet","type":"bool"}],"payable":false,"stateMutability":"view","type":"function","signature":"0xfbd9574d"},{"constant":true,"inputs":[{"name":"","type":"uint256"}],"name":"loanOrderHashes","outputs":[{"name":"","type":"bytes32"}],"payable":false,"stateMutability":"view","type":"function","signature":"0xfe056342"},{"payable":true,"stateMutability":"payable","type":"fallback"},{"anonymous":false,"inputs":[{"indexed":true,"name":"from","type":"address"},{"indexed":true,"name":"to","type":"address"},{"indexed":false,"name":"value","type":"uint256"}],"name":"Transfer","type":"event","signature":"0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"},{"anonymous":false,"inputs":[{"indexed":true,"name":"owner","type":"address"},{"indexed":true,"name":"spender","type":"address"},{"indexed":false,"name":"value","type":"uint256"}],"name":"Approval","type":"event","signature":"0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925"},{"anonymous":false,"inputs":[{"indexed":true,"name":"minter","type":"address"},{"indexed":false,"name":"tokenAmount","type":"uint256"},{"indexed":false,"name":"assetAmount","type":"uint256"},{"indexed":false,"name":"price","type":"uint256"}],"name":"Mint","type":"event","signature":"0xb4c03061fb5b7fed76389d5af8f2e0ddb09f8c70d1333abbb62582835e10accb"},{"anonymous":false,"inputs":[{"indexed":true,"name":"burner","type":"address"},{"indexed":false,"name":"tokenAmount","type":"uint256"},{"indexed":false,"name":"assetAmount","type":"uint256"},{"indexed":false,"name":"price","type":"uint256"}],"name":"Burn","type":"event","signature":"0x743033787f4738ff4d6a7225ce2bd0977ee5f86b91a902a58f5e4d0b297b4644"},{"anonymous":false,"inputs":[{"indexed":true,"name":"borrower","type":"address"},{"indexed":false,"name":"borrowAmount","type":"uint256"},{"indexed":false,"name":"interestRate","type":"uint256"},{"indexed":false,"name":"collateralTokenAddress","type":"address"},{"indexed":false,"name":"tradeTokenToFillAddress","type":"address"},{"indexed":false,"name":"withdrawOnOpen","type":"bool"}],"name":"Borrow","type":"event","signature":"0x86e15dd78cd784ab7788bcf5b96b9395e86030e048e5faedcfe752c700f6157e"},{"anonymous":false,"inputs":[{"indexed":true,"name":"claimant","type":"address"},{"indexed":false,"name":"tokenAmount","type":"uint256"},{"indexed":false,"name":"assetAmount","type":"uint256"},{"indexed":false,"name":"remainingTokenAmount","type":"uint256"},{"indexed":false,"name":"price","type":"uint256"}],"name":"Claim","type":"event","signature":"0x68e1caf97c4c29c1ac46024e9590f80b7a1f690d393703879cf66eea4e1e8421"},{"anonymous":false,"inputs":[{"indexed":true,"name":"previousOwner","type":"address"},{"indexed":true,"name":"newOwner","type":"address"}],"name":"OwnershipTransferred","type":"event","signature":"0x8be0079c531659141344cd1fd0a4f28419497f9722a3daafe3b4186f6b6457e0"},{"constant":false,"inputs":[{"name":"receiver","type":"address"}],"name":"mintWithEther","outputs":[{"name":"mintAmount","type":"uint256"}],"payable":true,"stateMutability":"payable","type":"function","signature":"0x8f6ede1f"},{"constant":false,"inputs":[{"name":"receiver","type":"address"},{"name":"depositAmount","type":"uint256"}],"name":"mint","outputs":[{"name":"mintAmount","type":"uint256"}],"payable":false,"stateMutability":"nonpayable","type":"function","signature":"0x40c10f19"},{"constant":false,"inputs":[{"name":"receiver","type":"address"},{"name":"burnAmount","type":"uint256"}],"name":"burnToEther","outputs":[{"name":"loanAmountPaid","type":"uint256"}],"payable":false,"stateMutability":"nonpayable","type":"function","signature":"0x81a6b250"},{"constant":false,"inputs":[{"name":"receiver","type":"address"},{"name":"burnAmount","type":"uint256"}],"name":"burn","outputs":[{"name":"loanAmountPaid","type":"uint256"}],"payable":false,"stateMutability":"nonpayable","type":"function","signature":"0x9dc29fac"},{"constant":false,"inputs":[{"name":"borrowAmount","type":"uint256"},{"name":"leverageAmount","type":"uint256"},{"name":"initialLoanDuration","type":"uint256"},{"name":"collateralTokenSent","type":"uint256"},{"name":"borrower","type":"address"},{"name":"collateralTokenAddress","type":"address"}],"name":"borrowTokenFromDeposit","outputs":[{"name":"loanOrderHash","type":"bytes32"}],"payable":true,"stateMutability":"payable","type":"function","signature":"0x985f7f3d"},{"constant":false,"inputs":[{"name":"borrowAmount","type":"uint256"},{"name":"leverageAmount","type":"uint256"},{"name":"interestInitialAmount","type":"uint256"},{"name":"loanTokenSent","type":"uint256"},{"name":"collateralTokenSent","type":"uint256"},{"name":"tradeTokenSent","type":"uint256"},{"name":"borrower","type":"address"},{"name":"collateralTokenAddress","type":"address"},{"name":"tradeTokenAddress","type":"address"}],"name":"borrowTokenAndUse","outputs":[{"name":"loanOrderHash","type":"bytes32"}],"payable":false,"stateMutability":"nonpayable","type":"function","signature":"0x2ddbfc19"},{"constant":false,"inputs":[{"name":"depositAmount","type":"uint256"},{"name":"leverageAmount","type":"uint256"},{"name":"loanTokenSent","type":"uint256"},{"name":"collateralTokenSent","type":"uint256"},{"name":"tradeTokenSent","type":"uint256"},{"name":"trader","type":"address"},{"name":"depositTokenAddress","type":"address"},{"name":"collateralTokenAddress","type":"address"},{"name":"tradeTokenAddress","type":"address"}],"name":"marginTradeFromDeposit","outputs":[{"name":"loanOrderHash","type":"bytes32"}],"payable":false,"stateMutability":"nonpayable","type":"function","signature":"0xfecb8da3"},{"constant":false,"inputs":[],"name":"claimLoanToken","outputs":[{"name":"claimedAmount","type":"uint256"}],"payable":false,"stateMutability":"nonpayable","type":"function","signature":"0x087fc48b"},{"constant":false,"inputs":[],"name":"wrapEther","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function","signature":"0x6f1296d2"},{"constant":false,"inputs":[{"name":"tokenAddress","type":"address"}],"name":"donateAsset","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function","signature":"0x59e239af"},{"constant":false,"inputs":[{"name":"_to","type":"address"},{"name":"_value","type":"uint256"}],"name":"transfer","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function","signature":"0xa9059cbb"},{"constant":false,"inputs":[{"name":"_from","type":"address"},{"name":"_to","type":"address"},{"name":"_value","type":"uint256"}],"name":"transferFrom","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function","signature":"0x23b872dd"},{"constant":true,"inputs":[],"name":"tokenPrice","outputs":[{"name":"price","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x7ff9b596"},{"constant":true,"inputs":[{"name":"_user","type":"address"}],"name":"checkpointPrice","outputs":[{"name":"price","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function","signature":"0xeebc5081"},{"constant":true,"inputs":[],"name":"totalReservedSupply","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x674d13c8"},{"constant":true,"inputs":[],"name":"marketLiquidity","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x612ef80b"},{"constant":true,"inputs":[],"name":"supplyInterestRate","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x09ec6b6b"},{"constant":true,"inputs":[],"name":"avgBorrowInterestRate","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x44a4a003"},{"constant":true,"inputs":[],"name":"borrowInterestRate","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x8325a1c0"},{"constant":true,"inputs":[{"name":"borrowAmount","type":"uint256"}],"name":"nextBorrowInterestRate","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function","signature":"0xb9fe1a8f"},{"constant":true,"inputs":[{"name":"borrowAmount","type":"uint256"}],"name":"nextLoanInterestRate","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function","signature":"0xac9fd2b8"},{"constant":true,"inputs":[{"name":"supplyAmount","type":"uint256"}],"name":"nextSupplyInterestRate","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function","signature":"0xd65a5021"},{"constant":true,"inputs":[],"name":"totalAssetSupply","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x8fb807c5"},{"constant":true,"inputs":[{"name":"leverageAmount","type":"uint256"}],"name":"getMaxEscrowAmount","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x829b38f4"},{"constant":true,"inputs":[],"name":"getLeverageList","outputs":[{"name":"","type":"uint256[]"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x2ecae90a"},{"constant":true,"inputs":[{"name":"_owner","type":"address"}],"name":"assetBalanceOf","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x06b3efd6"},{"constant":true,"inputs":[{"name":"borrowAmount","type":"uint256"},{"name":"leverageAmount","type":"uint256"},{"name":"initialLoanDuration","type":"uint256"},{"name":"collateralTokenAddress","type":"address"}],"name":"getDepositAmountForBorrow","outputs":[{"name":"depositAmount","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x8423acd6"},{"constant":true,"inputs":[{"name":"depositAmount","type":"uint256"},{"name":"leverageAmount","type":"uint256"},{"name":"initialLoanDuration","type":"uint256"},{"name":"collateralTokenAddress","type":"address"}],"name":"getBorrowAmountForDeposit","outputs":[{"name":"borrowAmount","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function","signature":"0x24d25f4a"},{"constant":true,"inputs":[{"name":"assetSupply","type":"uint256"}],"name":"_supplyInterestRate","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function","signature":"0xf468697e"},{"constant":false,"inputs":[{"components":[{"name":"loanTokenAddress","type":"address"},{"name":"interestTokenAddress","type":"address"},{"name":"collateralTokenAddress","type":"address"},{"name":"oracleAddress","type":"address"},{"name":"loanTokenAmount","type":"uint256"},{"name":"interestAmount","type":"uint256"},{"name":"initialMarginAmount","type":"uint256"},{"name":"maintenanceMarginAmount","type":"uint256"},{"name":"maxDurationUnixTimestampSec","type":"uint256"},{"name":"loanOrderHash","type":"bytes32"}],"name":"loanOrder","type":"tuple"},{"components":[{"name":"trader","type":"address"},{"name":"collateralTokenAddressFilled","type":"address"},{"name":"positionTokenAddressFilled","type":"address"},{"name":"loanTokenAmountFilled","type":"uint256"},{"name":"loanTokenAmountUsed","type":"uint256"},{"name":"collateralTokenAmountFilled","type":"uint256"},{"name":"positionTokenAmountFilled","type":"uint256"},{"name":"loanStartUnixTimestampSec","type":"uint256"},{"name":"loanEndUnixTimestampSec","type":"uint256"},{"name":"active","type":"bool"},{"name":"positionId","type":"uint256"}],"name":"loanPosition","type":"tuple"},{"name":"loanCloser","type":"address"},{"name":"closeAmount","type":"uint256"},{"name":"","type":"bool"}],"name":"closeLoanNotifier","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function","signature":"0xcd4fa66d"},{"constant":false,"inputs":[{"name":"settingsTarget","type":"address"},{"name":"txnData","type":"bytes"}],"name":"updateSettings","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function","signature":"0x284e2f56"}]'
bzx_valut_address = '0x8B3d70d628Ebd30D4A2ea82DB95bA2e906c71633'


compound_values = {
 'isCodeAudited': 1,
 'isCodeOpenSource': 1,
 'isCodeFormallyVerified': 1,
 'hasBugBounty': 1,
 'insuranceRisk': 0
}

dydx_values = {
 'isCodeAudited': 1,
 'isCodeOpenSource': 1,
 'isCodeFormallyVerified': 0,
 'hasBugBounty': 1,
 'insuranceRisk': 0
}

fulcrum_values = {
 'isCodeAudited': 1,
 'isCodeOpenSource': 1,
 'isCodeFormallyVerified': 0,
 'hasBugBounty': 0,
 'insuranceRisk': 0
}

weights = {
    'auditedCode': 0.25,
    'allCodeOSS': 0.15,
    'formalVer': 0.05,
    'hasBugBounty': 0.05,
    'cVaR': 0.1,
    'poolCollateralization': 0.15,
    'poolLiquidity': 0.1,
    'insuranceRisk': 0.15
}

compoundContractInfo = [
    {
      'token': 'eth',
      'contract': '0x4ddc2d193948926d02f9b1fe9e1daa0718270ed5'
    },
    {
      'token': 'dai',
      'contract': '0xf5dce57282a584d2746faf1593d3121fcac444dc'
    },
    {
      'token': 'usdc',
      'contract': '0x39aa39c021dfbae8fac545936693ac917d5e7563'
    },
    {
      'token': 'zrx',
      'contract': '0xb3319f5d18bc0d84dd1b4825dcde5d5f7266d407'
    },
    {
      'token': 'bat',
      'contract': '0x6c8c6b02e7b2be14d4fa6022dfd6d75921d90e4e'
    },
    {
      'token': 'rep',
      'contract': '0x158079ee67fce2f58472a96584a73c7ab9ac95c1'
    }
]

dydxContractInfo = {
    'contractAddress': '0x1E0447b19BB6EcFdAe1e4AE1694b0C3659614e4e',
    'markets': ['eth', 'dai', 'usdc']
}

fulcrumContractInfo = [
    {
        'token': 'dai',
        'contractAddress': '0x14094949152eddbfcd073717200da82fed8dc960',
        'baseTokenAddress': '0x89d24a6b4ccb1b6faa2625fe562bdd9a23260359'
    },
    {
        'token': 'eth',
        'contractAddress': '0x77f973fcaf871459aa58cd81881ce453759281bc',
        'baseTokenAddress': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'
    },
    {
        'token': 'usdc',
        'contractAddress': '0xf013406a0b1d544238083df0b93ad0d2cbe0f65f',
        'baseTokenAddress': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'
    },
    {
        'token': 'wbtc',
        'contractAddress': '0xba9262578efef8b3aff7f60cd629d6cc8859c8b5',
        'baseTokenAddress': '0x2260fac5e5542a773aa44fbcfedf7c193bc2c599'
    },
    {
        'token': 'rep',
        'contractAddress': '0xbd56e9477fc6997609cf45f84795efbdac642ff1',
        'baseTokenAddress': '0x1985365e9f78359a9B6AD760e32412f4a445E862'
    },
    {
        'token': 'link',
        'contractAddress': '0x1d496da96caf6b518b133736beca85d5c4f9cbc5',
        'baseTokenAddress': '0x514910771af9ca656af840dff83e8264ecf986ca'
    },
    {
        'token': 'zrx',
        'contractAddress': '0xA7Eb2bc82df18013ecC2A6C533fc29446442EDEe',
        'baseTokenAddress': '0xe41d2489571d322189246dafa5ebde1f4699f498'
    },
    {
        'token': 'knc',
        'contractAddress': '0x1cc9567ea2eb740824a45f8026ccf8e46973234d',
        'baseTokenAddress': '0xdd974d5c2e2928dea5f71b9825b8b646686bd200'
        
    },
]

# Used to find current USD values for all coins
def findCurrentUSDValue(token):
    tokenTicker = ''
    if (token[0] == 'i' and token[1] == 'w'):
        tokenTicker = token[2:].upper()
    elif (token[0] == 'c' or token[0] ==  'w' or token[0] == 'i'):
        tokenTicker = token[1:].upper()
    else:
        tokenTicker = token.upper()
    result = requests.get(f'https://min-api.cryptocompare.com/data/dayAvg?fsym={tokenTicker}&tsym=USD')
    json = result.json()
    return json['USD']

# Used to find balance for Compound Markets
def buildCompoundBalance(contract):
    shiftBy = 6 if contract['token'] == 'usdc' else 18
    abi = json.loads(compound_ceth_abi_string) if (contract['token'] == 'eth') else json.loads(erc20_abi_string)
    checksumAddress = w3.toChecksumAddress(contract['contract'])
    initializedContract = w3.eth.contract(address=checksumAddress, abi=abi)
    rawBalance = initializedContract.functions.getCash().call()
    balance = rawBalance / 10**shiftBy
    usdValue = findCurrentUSDValue(contract['token'])
    balanceValue = balance * usdValue
    contract['amount'] = balance
    contract['balance'] = balanceValue
    return contract

# Used to find balance for dydx Markets
def buildDydxBalance(market):
    shiftBy = 6 if market == 2 else 18
    token = dydxContractInfo['markets'][market]
    initializedContract = w3.eth.contract(dydxContractInfo['contractAddress'], abi=dydx_abi_string)
    marketData = initializedContract.functions.getMarketWithInfo(market).call()
    collateralAmountRaw = marketData[0][1][1] - marketData[0][1][0]
    collateralAmount = collateralAmountRaw / 10 ** shiftBy
    usdValue = findCurrentUSDValue(dydxContractInfo['markets'][market])
    collateralValue = collateralAmount * usdValue
    result = {}
    result['token'] = dydxContractInfo['markets'][market]
    result['amount'] = collateralAmount
    result['balance'] = collateralValue
    return result
   
def buildFulcrumBalance(contract):
    shiftBy = 0
    if (contract['token'] == 'wbtc'):
        shiftBy = 8
    elif (contract['token'] == 'usdc'):
        shiftBy = 6
    else:
        shiftBy = 18
    iTokenChecksumAddress = w3.toChecksumAddress(contract['contractAddress'])
    baseTokenChecksumAddress = w3.toChecksumAddress(contract['baseTokenAddress'])
    initializedContract = w3.eth.contract(address=iTokenChecksumAddress, abi=fulcrum_itoken_abi)
    baseTokenContract = w3.eth.contract(address=baseTokenChecksumAddress, abi=erc20_abi_string)
    iTokenBalance = initializedContract.functions.marketLiquidity().call()
    baseTokenBalance = baseTokenContract.functions.balanceOf(bzx_valut_address).call()
    rawBalance = iTokenBalance + baseTokenBalance
    balance = rawBalance / 10**shiftBy
    usdValue = findCurrentUSDValue(contract['token'])
    balanceValue = balance * usdValue
    contract['amount'] = balance
    contract['balance'] = balanceValue
    return contract

# Used to find portfolio weights for an array of balances
def getWeights(balances):
    total = 0
    weights = []
    for balance in balances:
        total += balance['balance']
    for balance in balances:
        percentage = balance['balance']/total
        weights.append(percentage)
    return weights

# Used to find historical USD values for stablecoins
def getCryptoCompareReturns(token):
    result = requests.get(f'https://min-api.cryptocompare.com/data/v2/histoday?fsym={token}&tsym=USD&limit=720')
    json = result.json()
    df = pd.DataFrame([object for object in json['Data']['Data']])
    df.insert(0, 'Date', pd.to_datetime(df['time'],unit='s'))
    df.drop(['high', 'open', 'low', 'volumefrom', 'volumeto', 'conversionType', 'conversionSymbol', 'time'], axis=1, inplace=True)
    df.set_index('Date', inplace=True)
    returns = df.copy().pct_change().fillna(value=0, axis=0).rename(columns={'close': f'daily_returns_{token}'})
    return returns
    
# Used to find historical USD values for all coins but stablecoins      
def getReturns(tokens): 
    first_date = '2017-10-06'
    last_date = '2019-09-25'
    df_list = []
    for token in tokens:
        if (token['token'][0] == 'i' and token['token'][1] == 'w'):
            token = token['token'][2:].upper()
        elif (token['token'][0] == 'c' or token['token'][0] ==  'w' or token['token'][0] == 'i'):
            token = token['token'][1:].upper()
        else:
            token = token['token'].upper()
        if (token == 'DAI' or token == 'USDC'):
            ticker_returns = getCryptoCompareReturns(token)
            df_list.append(ticker_returns)
        else:
            ticker = f'{token}-USD'
            ticker_close = pdr.get_data_yahoo(ticker, first_date, last_date)[['Close']]
            ticker_returns = ticker_close.copy().pct_change().fillna(value=0, axis=0).rename(columns={'Close': f'daily_returns_{token}'})
            df_list.append(ticker_returns)
    df = reduce(lambda x, y: pd.merge(x, y, on = 'Date'), df_list)
    return df

def value_at_risk(returns, weights, alpha=0.95, lookback_days=520):
    # Multiply asset returns by weights to get one weighted portfolio return
    portfolio_returns = returns.iloc[-lookback_days:].dot(weights)
    # Compute the correct percentile loss and multiply by value invested
    return np.percentile(portfolio_returns, 100 * (1-alpha))

def cvar(returns, weights, alpha=0.95, lookback_days=520):
    # Call out to our existing function
    var = value_at_risk(returns, weights, alpha, lookback_days=lookback_days)
    returns = returns.fillna(0.0)
    portfolio_returns = returns.iloc[-lookback_days:].dot(weights)
    # Get back to a return rather than an absolute loss
    var_pct_loss = var / 1
    return np.nanmean(portfolio_returns[portfolio_returns < var_pct_loss])

def EMACalc(m_array, m_range):
    k = 2/(m_range + 1)
    # first item is just the same as the first item in the input
    ema_array = [m_array[0]]
    # for the rest of the items, they are computed with the previous one
    i = 1
    while i < m_range:
        ema_array.append(m_array[i] * k + ema_array[i - 1] * (1 - k))
        i += 1
    return ema_array[len(ema_array) - 1];

In [9]:
compound_balances = list(map(buildCompoundBalance, compoundContractInfo))
compound_weights = getWeights(compound_balances)
compound_returns = getReturns(compound_balances)
compound_returns = compound_returns.fillna(0.0)
compound_portfolio_returns = compound_returns.fillna(0.0).iloc[-520:].dot(compound_weights)
compound_portfolio_var = value_at_risk(compound_returns, compound_weights, 0.99, 520)
compound_portfolio_cvar = cvar(compound_returns, compound_weights, 0.99, 520)
# add instead of subtract here because cvar from this function is negative
compound_values['cvar'] = 1 + compound_portfolio_cvar

In [29]:
dydx_balances = list(map(buildDydxBalance, [0, 1, 2]))
dydx_weights = getWeights(dydx_balances)
dydx_returns = getReturns(dydx_balances)
dydx_returns = dydx_returns.fillna(0.0)
dydx_portfolio_returns = dydx_returns.fillna(0.0).iloc[-520:].dot(dydx_weights)
dydx_portfolio_var = value_at_risk(dydx_returns, dydx_weights, 0.99, 520)
dydx_portfolio_cvar = cvar(dydx_returns, dydx_weights, 0.99, 520)
# add instead of subtract here because cvar from this function is negative
dydx_values['cvar'] = 1 + dydx_portfolio_cvar

In [30]:
fulcrum_balances = list(map(buildFulcrumBalance, fulcrumContractInfo))
fulcrum_weights = getWeights(fulcrum_balances)
fulcrum_returns = getReturns(fulcrum_balances)
fulcrum_returns = fulcrum_returns.fillna(0.0)
fulcrum_portfolio_returns = fulcrum_returns.fillna(0.0).iloc[-520:].dot(fulcrum_weights)
fulcrum_portfolio_var = value_at_risk(fulcrum_returns, fulcrum_weights, 0.99, 520)
fulcrum_portfolio_cvar = cvar(fulcrum_returns, fulcrum_weights, 0.99, 520)
# add instead of subtract here because cvar from this function is negative
fulcrum_values['cvar'] = 1 + fulcrum_portfolio_cvar

In [31]:
def normalize_data(val, list):
    max_value = max(list, key=lambda x: x['value'])['value']
    min_value = min(list, key=lambda x: x['value'])['value']
    return (val - min_value) / (max_value - min_value)

def get_collateralizations():
    collateralization_list = []
    start_date = datetime.datetime.now() + datetime.timedelta(-30)
    start_date_unix = round(time.mktime(start_date.timetuple()))
    data = requests.get(f'https://api.defi.alethio.tech/v0/stats?after={start_date_unix}&granularity=24').json()
    for market in data['data']:
        collateral_array = np.asarray([item[1] for item in market['collateral_ratio']], dtype=np.float64)
        log_collateral_ema = EMACalc(collateral_array, 30)
        market_name = f"{market['protocol']}-{market['asset']}"
        collateralization_list.append({ 'market': market_name, 'value': math.log(log_collateral_ema)  })
#         collateralization_object[f"{market['protocol']}-{market['asset']}"] = log_collateral_ema
    return collateralization_list

def get_liquidities():
#     liquidity_object = {}
    liquidity_list = []
    for balance in fulcrum_balances:
        market_name = f'bzx-{balance["token"]}'
        liquidity_list.append({ 'market': market_name, 'value': math.log(balance['balance']) })
    for balance in dydx_balances:
        market_name = f'dydx-{balance["token"]}'
        liquidity_list.append({ 'market': market_name, 'value': math.log(balance['balance']) })
    for balance in compound_balances:
        market_name = f'compound-{balance["token"]}'
        liquidity_list.append({ 'market': market_name, 'value': math.log(balance['balance']) })
    return liquidity_list

weights = {
    'auditedCode': 0.25,
    'allCodeOSS': 0.15,
    'formalVer': 0.05,
    'hasBugBounty': 0.05,
    'cVaR': 0.1,
    'poolCollateralization': 0.15,
    'poolLiquidity': 0.1,
    'insuranceRisk': 0.15
}

# TODO - make this function cleaner
def calculate_score(protocol, token):
    if protocol == 'dydx':
        protocol_values = dydx_values
    elif protocol == 'compound':
        protocol_values = compound_values
    else:
        protocol_values = fulcrum_values
    liquidity_array = get_liquidities()
    liquidity_value = next((x for x in liquidity_array if x['market'] == f'{protocol}-{token}'), None)['value']
    normalized_liquidity_value = normalize_data(liquidity_value, liquidity_array)
    collateralization_array = get_collateralizations()
    collateralization_value = next((x for x in collateralization_array if x['market'] == f'{protocol}-{token}'), None)['value']
    normalized_collateralization_value = normalize_data(liquidity_value, liquidity_array)
    # Actually calculate score
    score = weights['auditedCode'] * protocol_values['isCodeAudited'] + weights['allCodeOSS'] * protocol_values['isCodeOpenSource'] + weights['formalVer'] * protocol_values['isCodeFormallyVerified'] + weights['hasBugBounty'] * protocol_values['hasBugBounty'] + weights['cVaR'] * protocol_values['cvar'] + weights['poolCollateralization'] * normalized_collateralization_value + weights['poolLiquidity'] * normalized_liquidity_value
    score = round(score, 2) * 10
    return score

In [32]:
calculate_score('compound', 'eth')

8.4

In [33]:
calculate_score('compound', 'dai')

7.9

In [34]:
calculate_score('compound', 'bat')

7.4

In [35]:
calculate_score('compound', 'zrx')

7.4

In [36]:
calculate_score('compound', 'usdc')

8.0

In [37]:
calculate_score('compound', 'rep')

7.800000000000001

In [38]:
calculate_score('dydx', 'usdc')

6.7

In [39]:
calculate_score('dydx', 'dai')

7.0

In [40]:
calculate_score('dydx', 'eth')

7.6

In [41]:
calculate_score('bzx', 'eth')

6.3

In [42]:
calculate_score('bzx', 'dai')

6.2

In [43]:
calculate_score('bzx', 'usdc')

5.699999999999999

In [44]:
calculate_score('bzx', 'eth')

6.3

In [45]:
calculate_score('bzx', 'rep')

4.9

In [46]:
calculate_score('bzx', 'wbtc')

5.8

In [47]:
calculate_score('bzx', 'link')

5.8

In [48]:
calculate_score('bzx', 'knc')

5.2

In [49]:
calculate_score('bzx', 'zrx')

5.1